In [ ]:
import random
import time
import math
from hashlib import sha256


def coprime(a, b):
    while b != 0:
        a, b = b, a % b
    return a
    
    
def extended_gcd(a, b):
    if a == 0 :  
        return b, 0, 1
             
    gcd,x1,y1 = extended_gcd(b%a, a) 
     
    x = y1 - (b//a) * x1 
    y = x1 
     
    return gcd,x,y

#Thuật toán Euclid mở rộng  
def modinv(a, m):
	g, x, y = extended_gcd(a, m)
	if g != 1:
		raise Exception('Nghịch đảo mod không tồn tại')
	return x % m    

        
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True
    
def hashFunction(message):
  hashed = sha256(message.encode("UTF-8")).hexdigest()
  return hashed

# **Tạo khóa**: 
Hãy giúp Tâm sinh khóa để tạo chữ ký số.

In [ ]:
def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Cả 2 số p và q đều phải là số nguyên tố.')
    elif p == q:
        raise ValueError('2 số p và q phải khác nhau')

    N = p * q
    phi = (p-1) * (q-1)

    #Chọn số nguyên e sao cho e and phi(n) là 2 số nguyên tố cùng nhau
    e = random.randrange(1, phi)
    g = coprime(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = coprime(e, phi)

    #Sử dụng thuật toán Euclid mở rộng tìm d để tạo khóa Private
    d = modinv(e, phi)

    #Return khóa public và khóa Private
    return ((N, e), (N, d))

In [ ]:
p = int(input("Hãy nhập một số nguyên tố (17, 19, 23, v.vv...): "))
q = int(input("Hãy nhập một số nguyên tố khác (Không phải số đầu tiên): "))   

print()
print("Bắt đầu tạo khóa Public và Private . . .")

public, private = generate_keypair(p, q)
print("Khóa Public là: ", public ,"\nKhóa Private là: ", private)

Hãy nhập một số nguyên tố (17, 19, 23, v.vv...): 17
Hãy nhập một số nguyên tố khác (Không phải số đầu tiên): 23

Bắt đầu tạo khóa Public và Private . . .
Khóa Public là:  (391, 179) 
Khóa Private là:  (391, 59)


# **Tạo chữ kí số:** 
Hãy mã hóa và tạo chữ ký số với nội dung tin nhắn Tâm gửi "Lớp Toán tin K63".

In [ ]:
def encrypt(privatek, mabam_h):
    N, key = privatek

    #Chuyển đổi từng chữ cái trong bản rõ thành các số dựa trên bảng mã Unicode
    #và mã hóa bằng công thức s =  h ^ d mod N
            
    numberRepr = [ord(char) for char in mabam_h]
    print("Biểu diễn số của tin nhắn đã băm qua bản mã Unicode trước khi mã hóa: ")
    print(numberRepr)
    chukyso_s = [pow(ord(char),key,N) for char in mabam_h]
    
    #Return the array of bytes
    return chukyso_s

In [ ]:
message = 'Lớp Toán tin K63'
print("Tin nhắn cần gửi: ", message)
print()
print("Bắt đầu mã hóa tin nhắn với khóa bí mật ", private ," . . .")
mabam_h = hashFunction(message)
print("Tin nhắn sau được băm:", mabam_h)
chukyso_s = encrypt(private, mabam_h)
print("Chữ ký số sau khi đã được mã hóa bằng công thức s =  h ^ {} mod {}:".format(private[1], private[0]))
print(chukyso_s)

Tin nhắn cần gửi:  Lớp Toán tin K63

Bắt đầu mã hóa tin nhắn với khóa bí mật  (391, 59)  . . .
Tin nhắn sau được băm: 3c9433fb90d96c57d20d11654ee30053fca314f1b58469e2c5034ef45f35acdf
Biểu diễn số của tin nhắn đã băm qua bản mã Unicode trước khi mã hóa: 
[51, 99, 57, 52, 51, 51, 102, 98, 57, 48, 100, 57, 54, 99, 53, 55, 100, 50, 48, 100, 49, 49, 54, 53, 52, 101, 101, 51, 48, 48, 53, 51, 102, 99, 97, 51, 49, 52, 102, 49, 98, 53, 56, 52, 54, 57, 101, 50, 99, 53, 48, 51, 52, 101, 102, 52, 53, 102, 51, 53, 97, 99, 100, 102]
Chữ ký số sau khi đã được mã hóa bằng công thức s =  h ^ 59 mod 391:
[272, 129, 56, 307, 272, 272, 51, 123, 56, 384, 94, 56, 347, 129, 382, 98, 94, 118, 384, 94, 196, 196, 347, 382, 307, 305, 305, 272, 384, 384, 382, 272, 51, 129, 295, 272, 196, 307, 51, 196, 123, 382, 28, 307, 347, 56, 305, 118, 129, 382, 384, 272, 307, 305, 51, 307, 382, 51, 272, 382, 295, 129, 94, 51]


# **Kiểm tra chữ ký, xác thực tin nhắn:**
Sau khi gửi đi tin nhắn, giả sử Hiếu sẽ nhận được tin nhắn là "Lớp Toán tin K64" và chữ ký số s đi kèm. Hãy xác thực tin nhắn đó có phải đúng là Tâm gửi hay không?

In [ ]:
def decrypt(publick, received_sign):
    n, key = publick
       
    #sinh ra mã băm dựa trên chũ ký số 
    numberRepr = [pow(char, key, n) for char in received_sign]
    mabam_h = [chr(pow(char, key, n)) for char in received_sign]

    print("Biểu diễn số của mã băm được giải mã từ chữ ký số s được gửi tới: ")
    print(numberRepr)
    
    return ''.join(mabam_h)
  
def verify(receivedHashed, receivedmessage):
  ourHashed = hashFunction(receivedmessage)
  print("Mã băm của tin nhắn nhận được: ", ourHashed)
  if receivedHashed == ourHashed:
      print("Xác thực thành công: ", )
      print(receivedHashed, " = ", ourHashed)
  else:
      
      print("Xác thực thất bại: ")
      print(receivedHashed, " != ", ourHashed)

In [ ]:
receivedmessage = 'Lớp Toán tin K64'
received_sign = chukyso_s
print("Tin nhắn nhận được là: ", receivedmessage)
print("Chữ/ kí số nhận được là: ")
print(received_sign)
print()
receivedHashed = decrypt(public, received_sign)
print("Mã băm xong khi giải mã chữ ký số: ", receivedHashed)
print()
print("Bắt đầu xác thực . . .")
verify(receivedHashed, receivedmessage)

Tin nhắn nhận được là:  Lớp Toán tin K64
Chữ/ kí số nhận được là: 
[136, 181, 224, 358, 136, 136, 153, 174, 224, 96, 110, 224, 363, 181, 43, 81, 110, 16, 96, 110, 331, 331, 363, 43, 358, 288, 288, 136, 96, 96, 43, 136, 153, 181, 44, 136, 331, 358, 153, 331, 174, 43, 245, 358, 363, 224, 288, 16, 181, 43, 96, 136, 358, 288, 153, 358, 43, 153, 136, 43, 44, 181, 110, 153]

Biểu diễn số của mã băm được giải mã từ chữ ký số s được gửi tới: 
[51, 99, 57, 52, 51, 51, 102, 98, 57, 48, 100, 57, 54, 99, 53, 55, 100, 50, 48, 100, 49, 49, 54, 53, 52, 101, 101, 51, 48, 48, 53, 51, 102, 99, 97, 51, 49, 52, 102, 49, 98, 53, 56, 52, 54, 57, 101, 50, 99, 53, 48, 51, 52, 101, 102, 52, 53, 102, 51, 53, 97, 99, 100, 102]
Mã băm xong khi giải mã chữ ký số:  3c9433fb90d96c57d20d11654ee30053fca314f1b58469e2c5034ef45f35acdf

Bắt đầu xác thực . . .
Mã băm của tin nhắn nhận được:  276384310606a3b7aa7e82b0106732db22bad789e72b108de6c620639a9283d5
Xác thực thất bại: 
3c9433fb90d96c57d20d11654ee30053fca314f1b58469e2